<a href="https://colab.research.google.com/github/flor-g/Undergraduate-thesis/blob/main/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def solve_rsa_threshold(alpha=1.0, cost_all=0.0):
    """
    Determines the prior probability P(m_all) threshold at which the
    interpretation of 'some' flips from 'some-but-not-all' to 'all'.
    """

    # We iterate through possible Prior values for m_all (0.01 to 0.99)
    priors_all = np.linspace(0.001, 0.999, 1000)

    threshold = None

    for p_all in priors_all:
        p_sna = 1.0 - p_all  # Probability of Some-But-Not-All

        # --- 1. Literal Listener L0 (Uniform Prior) ---
        # We assume 2 states: [SNA, ALL]
        # Utterance 'some' is compatible with both {SNA, ALL}
        # Utterance 'all' is compatible only with {ALL}

        # L0(m | u='some') -> Uniform distribution over compatible states
        l0_some_given_all = 0.5
        l0_some_given_sna = 0.5

        # L0(m | u='all') -> 1.0 on ALL (0 on SNA)
        l0_all_given_all = 1.0

        # --- 2. Pragmatic Speaker S1 ---
        # Utility = ln(L0(m|u)) - cost

        # Scenario A: Speaker wants to communicate m_ALL
        # Option 1: Say 'some'
        # Utility = ln(0.5) ≈ -0.69
        u_some_given_all = np.log(l0_some_given_all)

        # Option 2: Say 'all'
        # Utility = ln(1.0) - cost = 0 - cost
        u_all_given_all = np.log(l0_all_given_all) - cost_all

        # Softmax: Calculate probability S1 chooses 'some' given state is ALL
        # P(u_some | m_all) \propto exp(alpha * Utility)
        score_some = np.exp(alpha * u_some_given_all)
        score_all = np.exp(alpha * u_all_given_all)
        s1_some_given_all = score_some / (score_some + score_all)

        # Scenario B: Speaker wants to communicate m_SNA
        # Must say 'some' (assuming 'all' is false/impossible)
        s1_some_given_sna = 1.0

        # --- 3. Pragmatic Listener L1 ---
        # Bayes Rule: P(m | u) \propto S1(u | m) * P(m)

        # Evidence for ALL comes from the Speaker's likelihood * Prior
        posterior_unnorm_all = s1_some_given_all * p_all

        # Evidence for SNA comes from Speaker's likelihood * Prior
        posterior_unnorm_sna = s1_some_given_sna * p_sna

        # Normalize
        prob_all_given_some = posterior_unnorm_all / (posterior_unnorm_all + posterior_unnorm_sna)

        # --- 4. Check Threshold ---
        # Does the "non-scalar" reading (m_all) prevail?
        if prob_all_given_some > 0.5:
            threshold = p_all
            break

    return threshold

# --- Run the Model ---
alphas = [1.0, 2.0, 3.0, 4.0]
print(f"{'Alpha':<10} | {'Threshold P(m_all)':<20}")
print("-" * 35)

for a in alphas:
    t = solve_rsa_threshold(alpha=a)
    print(f"{a:<10} | {t:.4f}")

Alpha      | Threshold P(m_all)  
-----------------------------------
1.0        | 0.7502
2.0        | 0.8342
3.0        | 0.9001
4.0        | 0.9451
